In [ ]:
#| default_exp text

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#|hide
from nbdev import nbdev_export
from fastcore.test import test_eq, test_fail

# text
> Reference API related to text applications

In [ ]:
#|export
import sys
from os import PathLike
from typing import List, Optional, Union, Mapping, Dict, IO
from pathlib import Path
from urllib.parse import urlencode
from fastcore.utils import patch, patch_to

from vespa.package import (
    ModelConfig,
    Task,
    OnnxModel,
    QueryTypeField,
    Field,
    Function,
    RankProfile,
)
from vespa.json_serialization import ToJson, FromJson

#
# Optional ML dependencies
#
try:
    from torch import tensor
    from transformers import (
        AutoTokenizer,
        AutoModelForSequenceClassification,
        BertForSequenceClassification,
        BertTokenizerFast,
        pipeline,
    )
    from transformers.convert_graph_to_onnx import convert_pytorch
except ModuleNotFoundError:
    raise Exception("Use pip install pyvespa[ml] to install ml dependencies.")


/Users/tmartins/.local/share/virtualenvs/learntorank-qPFjAqDO/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-01 11:57:56.141784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#|export
class TextTask(Task):
    def __init__(
        self,
        model_id: str,  # Id used to identify the model on Vespa applications.
        model: str,  # Id of the model as used by the model hub.
        tokenizer: Optional[str] = None,  # Id of the tokenizer as used by the model hub.
        output_file: IO = sys.stdout,  # Output file to write output messages.
    ):
        "Base class for Tasks involving text inputs."
        super().__init__(model_id=model_id)
        self.model = model
        self.tokenizer = tokenizer
        if not self.tokenizer:
            self.tokenizer = model
        self.output = output_file
        self._tokenizer = None
        self._model = None

    @staticmethod
    def parse_vespa_prediction(prediction) -> List:
        return prediction["values"][0]

In [ ]:
#|export
@patch
def _load_tokenizer(self: TextTask):
    if not self._tokenizer:
        print("Downloading tokenizer.", file=self.output)
        self._tokenizer = AutoTokenizer.from_pretrained(self.tokenizer)

In [ ]:
#|export
@patch
def _create_pipeline(self: TextTask):
    self._load_tokenizer()
    if not self._model:
        print("Downloading model.", file=self.output)
        self._model = AutoModelForSequenceClassification.from_pretrained(self.model)
        print("Model loaded.", file=self.output)

    _pipeline = pipeline(
        task="text-classification",
        model=self._model,
        tokenizer=self._tokenizer,
        top_k=2,
        function_to_apply="None",
    )
    return _pipeline

In [ ]:
#|export
@patch
def export_to_onnx(
    self: TextTask, 
    output_path: str  # Relative output path for the onnx model, should end in '.onnx'
) -> None:
    "Export a model to ONNX"
    pipeline = self._create_pipeline()
    convert_pytorch(
        pipeline, opset=11, output=Path(output_path), use_external_format=False
    )


In [ ]:
#|export
@patch
def predict(
    self: TextTask, 
    text: str  # text input for the task.
) -> List:  # Predictions.
    "Predict using a local instance of the model"
    pipeline = self._create_pipeline()
    predictions = pipeline(text)
    return [
        [x["score"] for x in predictions[0] if x["label"] == "LABEL_0"][0],
        [x["score"] for x in predictions[0] if x["label"] == "LABEL_1"][0],
    ]


In [ ]:
#|export
@patch
def create_url_encoded_tokens(self: TextTask, x):
    raise NotImplementedError

In [ ]:
#|export
class SequenceClassification(TextTask):
    def __init__(
        self,
        model_id: str,  # Id used to identify the model on Vespa applications.
        model: str,  # Id of the model as used by the model hub. Alternatively, it can also be the path to the folder containing the model files, as long as the model config is also there.
        tokenizer: Optional[str] = None,  # Id of the tokenizer as used by the model hub. Alternatively, it can also be the path to the folder containing the tokenizer files, as long as the model config is also there.
        output_file: IO = sys.stdout,  # Output file to write output messages.
    ):
        """
        Sequence Classification task.

        It takes a text input and returns an array of floats depending on which
        model is used to solve the task.
        """
        super().__init__(
            model_id=model_id, model=model, tokenizer=tokenizer, output_file=output_file
        )


In [ ]:
#|export
@patch
def create_url_encoded_tokens(
    self: SequenceClassification, 
    x
):
    self._load_tokenizer()
    tokens = self._tokenizer(x)
    encoded_tokens = urlencode(
        {
            key: "{"
            + ",".join(
                [
                    "{{d0: 0, d1: {}}}: {}".format(idx, x)
                    for idx, x in enumerate(value)
                ]
            )
            + "}"
            for key, value in tokens.items()
        }
    )
    return encoded_tokens

In [ ]:
#|export
class BertModelConfig(ModelConfig):
    def __init__(
        self,
        model_id: str,  # Unique model id to represent the model within a Vespa application.
        query_input_size: int,  # The size of the input vector dedicated to the query text.
        doc_input_size: int,  # The size of the input vector dedicated to the document text.
        tokenizer: Union[str, PathLike],  # The name or a path to a saved BERT model tokenizer from the transformers library.
        model: Optional[Union[str, PathLike]] = None,  # The name or a path to a saved model that is compatible with the `tokenizer`. The model is optional at construction since you might want to train it first. You must add a model via :func:`add_model` before deploying a Vespa application that uses this class.
    ) -> None:
        """
        BERT model configuration for Vespa applications.


        >>> bert_config = BertModelConfig(
        ...     model_id="pretrained_bert_tiny",
        ...     query_input_size=32,
        ...     doc_input_size=96,
        ...     tokenizer="google/bert_uncased_L-2_H-128_A-2",
        ...     model="google/bert_uncased_L-2_H-128_A-2",
        ... )  # doctest: +SKIP
        BertModelConfig('pretrained_bert_tiny', 32, 96, 'google/bert_uncased_L-2_H-128_A-2', 'google/bert_uncased_L-2_H-128_A-2')
        """
        super().__init__(model_id=model_id)

        self.query_input_size = query_input_size
        self.doc_input_size = doc_input_size
        self.input_size = self.query_input_size + self.doc_input_size

        self.tokenizer = tokenizer
        self._tokenizer = BertTokenizerFast.from_pretrained(tokenizer)
        self._validate_tokenizer()

        self.query_token_ids_name = model_id + "_query_token_ids"
        self.actual_query_input_size = (
            query_input_size - 2
        )  # one character saved for CLS and one for SEP

        self.doc_token_ids_name = model_id + "_doc_token_ids"
        self.actual_doc_input_size = doc_input_size - 1  # one character saved for SEP

        self.model = model
        self._model = None
        if model:
            self.add_model(model=model)


In [ ]:
#bert_config = BertModelConfig(
#    model_id="pretrained_bert_tiny",
#    query_input_size=32,
#    doc_input_size=96,
#    tokenizer="google/bert_uncased_L-2_H-128_A-2"
#)

In [ ]:
#bert_config = BertModelConfig(
#    model_id="pretrained_bert_tiny",
#    query_input_size=32,
#    doc_input_size=96,
#    tokenizer="google/bert_uncased_L-2_H-128_A-2",
#    model="google/bert_uncased_L-2_H-128_A-2",
#)

In [ ]:
#|export
@patch
def predict(
    self: BertModelConfig, 
    queries,  # A List of query texts. 
    docs  # A List of document texts.
) -> List:  # Logits
    "Predict (forward pass) given queries and docs texts"

    if not self._model:
        raise ValueError("A model needs to be added.")
    model_output = self._model(
        **self.create_encodings(queries=queries, docs=docs, return_tensors=True),
        return_dict=True
    )
    return model_output.logits.tolist()

In [ ]:
#|export
@patch
def _validate_tokenizer(self:BertModelConfig) -> None:
    dummy_inputs = self._generate_dummy_inputs()

    assert (
        dummy_inputs["input_ids"].shape[1]
        == dummy_inputs["token_type_ids"].shape[1]
        and dummy_inputs["token_type_ids"].shape[1]
        == dummy_inputs["attention_mask"].shape[1]
        and dummy_inputs["attention_mask"].shape[1] == self.input_size
    ), "tokenizer generates wrong input size"

In [ ]:
#|export
@patch
def _validate_model(self: BertModelConfig, model: BertForSequenceClassification) -> None:
    if not isinstance(model, BertForSequenceClassification):
        raise ValueError("We only support BertForSequenceClassification for now.")
    model_output = model(**self._generate_dummy_inputs(), return_dict=True)
    if len(model_output.logits.shape) != 2:
        ValueError("Model output expected to be logits vector of size 2")


In [ ]:
#|export
@patch
def add_model(
    self: BertModelConfig, 
    model: Union[str, PathLike]  # The name or a path to a saved model that is compatible with the `tokenizer`.
) -> None:
    "Add a BERT model"
    
    _model = BertForSequenceClassification.from_pretrained(model)
    self._validate_model(model=_model)
    self.model = model
    self._model = _model

In [ ]:
#|export
@patch
def _query_input_ids(self: BertModelConfig, queries: List[str]):
    queries_encodings = self._tokenizer(
        queries,
        truncation=True,
        max_length=self.query_input_size - 2,
        add_special_tokens=False,
    )
    return queries_encodings["input_ids"]

In [ ]:
#|export
@patch
def _doc_input_ids(self: BertModelConfig, docs: List[str]):
    docs_encodings = self._tokenizer(
        docs,
        truncation=True,
        max_length=self.doc_input_size - 1,
        add_special_tokens=False,
    )
    return docs_encodings["input_ids"]

In [ ]:
#|export
@patch
def doc_fields(
    self: BertModelConfig, 
    text: str  # The text related to the document to be used as input to the bert model
) -> Dict:  # Dict with key and values as expected by Vespa.
    "Generate document fields related to the model that needs to be fed to Vespa."
    
    input_ids = self._doc_input_ids([text])[0]
    if len(input_ids) < self.actual_doc_input_size:
        input_ids = input_ids + [0] * (self.actual_doc_input_size - len(input_ids))
    return {self.doc_token_ids_name: {"values": input_ids}}

In [ ]:
#|export
@patch
def query_tensor_mapping(
    self: BertModelConfig, 
    text: str # Query text to be used as input to the BERT model.
) -> List[float]:  # Input ids expected by Vespa.
    "Maps query text to a tensor expected by Vespa at run time."
    
    input_ids = self._query_input_ids([text])[0]
    if len(input_ids) < self.actual_query_input_size:
        input_ids = input_ids + [0] * (
            self.actual_query_input_size - len(input_ids)
        )
    return input_ids

In [ ]:
#|export
@patch
def create_encodings(
    self: BertModelConfig, 
    queries: List[str], # Query texts.
    docs: List[str],  # Document texts. 
    return_tensors=False  # Return tensors
) -> Dict:  # Dict containing `input_ids`, `token_type_ids` and `attention_mask` encodings.
    """
    Create BERT model encodings.

    Create BERT encodings following the same pattern used during Vespa serving. Useful to generate training data
    and ensuring training and serving compatibility.
    """
    query_input_ids = self._query_input_ids(queries=queries)
    doc_input_ids = self._doc_input_ids(docs=docs)

    TOKEN_NONE = 0
    TOKEN_CLS = 101
    TOKEN_SEP = 102

    input_ids = []
    token_type_ids = []
    attention_mask = []
    for query_input_id, doc_input_id in zip(query_input_ids, doc_input_ids):
        # create input id
        input_id = (
            [TOKEN_CLS] + query_input_id + [TOKEN_SEP] + doc_input_id + [TOKEN_SEP]
        )
        number_tokens = len(input_id)
        padding_length = max(self.input_size - number_tokens, 0)
        input_id = input_id + [TOKEN_NONE] * padding_length
        input_ids.append(input_id)
        # create token id
        token_type_id = (
            [0] * len([TOKEN_CLS] + query_input_id + [TOKEN_SEP])
            + [1] * len(doc_input_id + [TOKEN_SEP])
            + [TOKEN_NONE] * padding_length
        )
        token_type_ids.append(token_type_id)
        # create attention_mask
        attention_mask.append([1] * number_tokens + [TOKEN_NONE] * padding_length)

    if return_tensors:
        input_ids = tensor(input_ids)
        token_type_ids = tensor(token_type_ids)
        attention_mask = tensor(attention_mask)
    encodings = {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask,
    }
    return encodings


In [ ]:
#|export
@patch
def _generate_dummy_inputs(self:BertModelConfig):
    dummy_input = self.create_encodings(
        queries=["dummy query 1"], docs=["dummy document 1"], return_tensors=True
    )
    return dummy_input


In [ ]:
#|export
@patch
def export_to_onnx(
    self: BertModelConfig, 
    output_path: str  # Relative output path for the onnx model, should end in '.onnx'
) -> None:
    "Export a model to ONNX"

    if self._model:
        _pipeline = pipeline(
            task="text-classification",
            model=self._model,
            tokenizer=self._tokenizer,
            return_all_scores=True,
            function_to_apply="None",
        )
        convert_pytorch(
            _pipeline, opset=11, output=Path(output_path), use_external_format=False
        )
    else:
        raise ValueError("No BERT model found to be exported.")


In [ ]:
#|export
@patch
def onnx_model(self: BertModelConfig):
    model_file_path = self.model_id + ".onnx"
    self.export_to_onnx(output_path=model_file_path)

    return OnnxModel(
        model_name=self.model_id,
        model_file_path=model_file_path,
        inputs={
            "input_ids": "input_ids",
            "token_type_ids": "token_type_ids",
            "attention_mask": "attention_mask",
        },
        outputs={"output_0": "logits"},
    )


In [ ]:
#|export
@patch
def query_profile_type_fields(self: BertModelConfig):
    return [
        QueryTypeField(
            name="ranking.features.query({})".format(self.query_token_ids_name),
            type="tensor<float>(d0[{}])".format(int(self.actual_query_input_size)),
        )
    ]

In [ ]:
#|export
@patch
def document_fields(self: BertModelConfig, document_field_indexing):
    if not document_field_indexing:
        document_field_indexing = ["attribute", "summary"]

    return [
        Field(
            name=self.doc_token_ids_name,
            type="tensor<float>(d0[{}])".format(int(self.actual_doc_input_size)),
            indexing=document_field_indexing,
        ),
    ]

In [ ]:
#|export
@patch
def rank_profile(self: BertModelConfig, include_model_summary_features, **kwargs):
    constants = {"TOKEN_NONE": 0, "TOKEN_CLS": 101, "TOKEN_SEP": 102}
    # ToDo: Add a unit test for constants
    if "constants" in kwargs:
        constants.update(kwargs.pop("constants"))

    functions = [
        Function(
            name="question_length",
            expression="sum(map(query({}), f(a)(a > 0)))".format(
                self.query_token_ids_name
            ),
        ),
        Function(
            name="doc_length",
            expression="sum(map(attribute({}), f(a)(a > 0)))".format(
                self.doc_token_ids_name
            ),
        ),
        Function(
            name="input_ids",
            expression="tokenInputIds({}, query({}), attribute({}))".format(
                self.input_size,
                self.query_token_ids_name,
                self.doc_token_ids_name,
            ),
        ),
        Function(
            name="attention_mask",
            expression="tokenAttentionMask({}, query({}), attribute({}))".format(
                self.input_size,
                self.query_token_ids_name,
                self.doc_token_ids_name,
            ),
        ),
        Function(
            name="token_type_ids",
            expression="tokenTypeIds({}, query({}), attribute({}))".format(
                self.input_size,
                self.query_token_ids_name,
                self.doc_token_ids_name,
            ),
        ),
        Function(
            name="logit0",
            expression="onnx(" + self.model_id + ").logits{d0:0,d1:0}",
        ),
        Function(
            name="logit1",
            expression="onnx(" + self.model_id + ").logits{d0:0,d1:1}",
        ),
    ]
    if "functions" in kwargs:
        functions.extend(kwargs.pop("functions"))

    summary_features = []
    if include_model_summary_features:
        summary_features.extend(
            [
                "logit0",
                "logit1",
                "input_ids",
                "attention_mask",
                "token_type_ids",
            ]
        )
    if "summary_features" in kwargs:
        summary_features.extend(kwargs.pop("summary_features"))

    return RankProfile(
        name=self.model_id,
        constants=constants,
        functions=functions,
        summary_features=summary_features,
        **kwargs
    )


In [ ]:
#|export
@patch
def __eq__(self: BertModelConfig, other):
    if not isinstance(other, self.__class__):
        return False
    return (
        self.model_id == other.model_id
        and self.query_input_size == other.query_input_size
        and self.doc_input_size == other.doc_input_size
        and self.tokenizer == other.tokenizer
        and self.model == other.model
    )

In [ ]:
#|export
@patch
def __repr__(self: BertModelConfig):
    return "{0}({1}, {2}, {3}, {4}, {5})".format(
        self.__class__.__name__,
        repr(self.model_id),
        repr(self.query_input_size),
        repr(self.doc_input_size),
        repr(self.tokenizer),
        repr(self.model),
    )

In [ ]:
#|hide
nbdev_export()